In [57]:
import logging
import os
from typing import List

import pandas as pd
from langchain import PromptTemplate
from langchain.chains import QAGenerationChain
from langchain.chat_models import AzureChatOpenAI
from langchain.document_loaders import TextLoader
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.evaluation import EmbeddingDistance
from langchain.evaluation import load_evaluator, EvaluatorType
from langchain.evaluation.schema import StringEvaluator
from langchain.retrievers.multi_query import MultiQueryRetriever
from langchain.schema import BaseRetriever
from langchain.schema import Document
from langchain.vectorstores import FAISS

from src.prompt_templates import MULTI_QA_GPT35_PROMPT_TEMPLATE, GRADE_DOCS_PROMPT_TEMPLATE

In [58]:
root_dir = "../linux-kernel"

docs = []
for dirpath, dirnames, filenames in os.walk(root_dir):
    for file in filenames:
        if file.endswith(".c") and "/.venv/" not in dirpath:
            try:
                loader = TextLoader(os.path.join(dirpath, file), encoding="utf-8")
                docs.extend(loader.load())
            except Exception as e:
                pass
print(f"{len(docs)}")

4


In [59]:
from langchain.text_splitter import (
    RecursiveCharacterTextSplitter,
    Language,
)

RecursiveCharacterTextSplitter.get_separators_for_language(Language.CPP)

cpp_splitter = RecursiveCharacterTextSplitter.from_language(
    language=Language.CPP, chunk_size=250, chunk_overlap=50
)
chunks = cpp_splitter.split_documents(docs)
print(f"{len(chunks)}")

1227


In [60]:
hf_embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-mpnet-base-v2",
    model_kwargs={'device': 'cuda'},
    encode_kwargs={'normalize_embeddings': False}
)

In [61]:
from langchain.embeddings.openai import OpenAIEmbeddings

openai_azure_embeddings = OpenAIEmbeddings(
    openai_api_key="02e3dbabaf334ccb959cbeadbd3f99c3",
    openai_api_base="https://llm-x-gpt.openai.azure.com/",
    deployment_name='LLM-X-Embedding',
    openai_api_version="2023-03-15-preview"
)

/home/watson/.virtualenvs/QnAEvaluation/lib/python3.9/site-packages/langchain/embeddings/openai.py:214: UserWarning: WARNING! deployment_name is not default parameter.
                    deployment_name was transferred to model_kwargs.
                    Please confirm that deployment_name is what you intended.
  warnings.warn(


In [62]:
hf_embeddings_vector_db = FAISS.from_documents(chunks, hf_embeddings)
hf_embeddings_vector_db.save_local("linux-kernel_embeddings")

In [63]:
question = "What is the purpose of the get_user_page in the linux kernel?"
retrieved_chunks = hf_embeddings_vector_db.similarity_search(question)
for i, retrieved_chunk in enumerate(retrieved_chunks):
    print(f"\n\n\ndoc {i}:\n {retrieved_chunk.page_content}")




doc 0:
 *
 * __get_user_pages walks a process's page tables and takes a reference to
 * each struct page that each user address corresponds to at a given
 * instant. That is, it takes the page that would be accessed if a user



doc 1:
 * trying again.
 *
 * Typically this is meant to be used by the futex code.
 *
 * The main difference with get_user_pages() is that this function will
 * unconditionally call handle_mm_fault() which will in turn perform all the



doc 2:
 *
 * get_user_pages_remote walks a process's page tables and takes a reference
 * to each struct page that each user address corresponds to at a given
 * instant. That is, it takes the page that would be accessed if a user



doc 3:
 *
 * get_user_pages_remote is typically used for fewer-copy IO operations,
 * to get a handle on the memory by some means other than accesses
 * via the user virtual addresses. The pages may be submitted for


In [8]:
gpt35_azure_llm = AzureChatOpenAI(
    temperature=0,
    openai_api_key="02e3dbabaf334ccb959cbeadbd3f99c3",
    openai_api_base="https://llm-x-gpt.openai.azure.com/",
    deployment_name='LLM-X-GPT35-TURBO',
    openai_api_version="2023-03-15-preview"
)

In [32]:
gpt4_azure_llm = AzureChatOpenAI(
    temperature=0,
    openai_api_key="a8d69f68a36b40789df2cc3fdbaacda9",
    openai_api_base="https://llmx-gpt-canada-east.openai.azure.com/",
    deployment_name='LLM-X-GPT-4',
    openai_api_version="2023-03-15-preview"
)

In [12]:
from langchain.chains import RetrievalQA

qa_rag_chain = RetrievalQA.from_chain_type(
    llm=gpt35_azure_llm,
    retriever=hf_embeddings_vector_db.as_retriever()
)

qa_rag_chain({"query": question})

{'query': 'What is the purpose of the get_user_page in the linux kernel?',
 'result': "The purpose of the get_user_pages() function in the Linux kernel is to pin user pages in memory. It allows the kernel to obtain references to the physical pages that correspond to a given user virtual address. This is useful when the kernel needs to access or manipulate user memory, such as when submitting pages for DMA to devices or accessing them via the kernel linear mapping. The function ensures that the pages won't be freed completely and guarantees that the page contains data that was valid at some point in time. The caller is responsible for releasing the returned pages using put_page()."}

In [13]:
multi_qa_prompt = PromptTemplate.from_template(template=MULTI_QA_GPT35_PROMPT_TEMPLATE,
                                               partial_variables={"k": 2})
qa_generation_chain = QAGenerationChain.from_llm(llm=gpt35_azure_llm,
                                                 prompt=multi_qa_prompt)

In [15]:
logging.basicConfig()
logging.getLogger('langchain.retrievers.multi_query').setLevel(logging.INFO)
retriever_from_llm = MultiQueryRetriever.from_llm(
    retriever=hf_embeddings_vector_db.as_retriever(),
    llm=gpt35_azure_llm
)

unique_docs = retriever_from_llm.get_relevant_documents(query=question)

INFO:langchain.retrievers.multi_query:Generated queries: ['1. What is the function of the get_user_page in the linux kernel?', '2. How does the get_user_page function serve its purpose in the linux kernel?', '3. Can you explain the role and significance of the get_user_page in the linux kernel?']


In [37]:
def create_qna_GT_df(docs: List[Document], num_of_qna_for_doc: int) -> pd.DataFrame:
    multi_qa_prompt = PromptTemplate.from_template(template=MULTI_QA_GPT35_PROMPT_TEMPLATE,
                                                   partial_variables={"k": num_of_qna_for_doc})
    qa_generation_chain = QAGenerationChain.from_llm(llm=gpt35_azure_llm,
                                                     prompt=multi_qa_prompt)

    qna_GT = []
    for doc in docs:
        doc_qna = qa_generation_chain.run(doc.page_content)
        qna_GT += doc_qna

    qna_GT_df = pd.DataFrame(qna_GT)
    return qna_GT_df

In [42]:
def get_qna_with_chain_answers_df(qa_rag_chain: RetrievalQA, qna_GT_df: pd.DataFrame) -> pd.DataFrame:
    qna_with_chain_answers = qna_GT_df.copy()
    qna_with_chain_answers["chain_answer"] = qna_with_chain_answers.apply(
        lambda qna: qa_rag_chain({"query": qna["question"]})["result"], axis=1)

    return qna_with_chain_answers

In [43]:
def get_evaluator_score(evaluator: StringEvaluator, qna_with_chain_answer: pd.Series) -> float:
    grade = evaluator.evaluate_strings(
        prediction=qna_with_chain_answer["chain_answer"],
        reference=qna_with_chain_answer["answer"],
        input=qna_with_chain_answer["question"])

    return grade["score"]

In [45]:
def get_retrieval_score(retriever, qna_with_chain_answer: pd.Series):
    GRADE_DOCS_PROMPT = PromptTemplate(input_variables=['result', 'answer', 'query'],
                                       template=GRADE_DOCS_PROMPT_TEMPLATE)
    retrieval_eval_chain = load_evaluator(
        evaluator=EvaluatorType.QA,
        llm=gpt35_azure_llm,
        prompt=GRADE_DOCS_PROMPT
    )

    retrieved_docs = retriever.get_relevant_documents(query=qna_with_chain_answer["question"],
                                                      search_type="similarity_score_threshold",
                                                      search_kwargs={"k": 2})

    grade = retrieval_eval_chain.evaluate_strings(
        prediction=retrieved_docs,
        reference=qna_with_chain_answer["answer"],
        input=qna_with_chain_answer["question"])

    return grade["score"]

In [46]:
def get_grades_for_chain_qna(qna_with_chain_answers_df: pd.DataFrame,
                             retriever: BaseRetriever = None) -> pd.DataFrame:
    grades_for_chain_qna = qna_with_chain_answers_df.copy()

    labeled_criteria_evaluator = load_evaluator(evaluator=EvaluatorType.LABELED_CRITERIA,
                                                criteria="correctness",
                                                llm=gpt35_azure_llm)

    embedding_distance_evaluator = load_evaluator(evaluator=EvaluatorType.EMBEDDING_DISTANCE,
                                                  distance_metric=EmbeddingDistance.COSINE,
                                                  embeddings=hf_embeddings,
                                                  llm=gpt35_azure_llm)

    qa_llm_jugde_evaluator = load_evaluator(evaluator=EvaluatorType.QA,
                                            llm=gpt35_azure_llm)

    grades_for_chain_qna["labeled_criteria_grades"] = qna_with_chain_answers_df.apply(
        lambda qna_with_chain_answer: get_evaluator_score(
            evaluator=labeled_criteria_evaluator,
            qna_with_chain_answer=qna_with_chain_answer), axis=1)

    grades_for_chain_qna["embedding_distance_grades"] = qna_with_chain_answers_df.apply(
        lambda qna_with_chain_answer: get_evaluator_score(
            evaluator=embedding_distance_evaluator,
            qna_with_chain_answer=qna_with_chain_answer), axis=1)

    grades_for_chain_qna["qa_llm_jugde_grades"] = qna_with_chain_answers_df.apply(
        lambda qna_with_chain_answer: get_evaluator_score(
            evaluator=qa_llm_jugde_evaluator,
            qna_with_chain_answer=qna_with_chain_answer), axis=1)

    if retriever:
        grades_for_chain_qna["retrieval_score"] = qna_with_chain_answers_df.apply(
            lambda qna_with_chain_answer: get_retrieval_score(
                retriever=retriever,
                qna_with_chain_answer=qna_with_chain_answer), axis=1)

    return grades_for_chain_qna

In [47]:
def run_evaluation(qa_rag_chain: RetrievalQA, qna_GT_df: pd.DataFrame) -> pd.DataFrame:
    qna_with_chain_answers_df = get_qna_with_chain_answers_df(qa_rag_chain, qna_GT_df)
    grades_for_chain_qna = \
        get_grades_for_chain_qna(qna_with_chain_answers_df=qna_with_chain_answers_df,
                                 retriever=qa_rag_chain.retriever)
    return grades_for_chain_qna

In [17]:
qna_GT_df = create_qna_GT_df(docs=docs, num_of_qna_for_doc=2)

In [78]:
langchain_qa_generation_chain = QAGenerationChain.from_llm(llm=gpt4_azure_llm)

In [86]:
from src.prompt_templates import MULTI_QA_GPT4_PROMPT_TEMPLATE

multi_qa_prompt = PromptTemplate.from_template(template=MULTI_QA_GPT4_PROMPT_TEMPLATE,
                                                   partial_variables={"k": 3})
qa_generation_chain = QAGenerationChain.from_llm(llm=gpt4_azure_llm,
                                                prompt=multi_qa_prompt)

In [85]:
doc = docs[0]
len(doc.page_content)

37203

In [83]:
doc_qna = langchain_qa_generation_chain.run(doc.page_content[:500])

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [95]:
doc_qna = qa_generation_chain.run(doc.page_content[:15000])

In [99]:
qnas_in_doc = []
for qna in doc_qna:
    qnas_in_doc += qna

In [100]:
qnas_in_doc

[{'question': 'What does the module implemented in the given code do?',
  'answer': 'The module implements the Address Resolution Protocol ARP (RFC 826), which is used to convert IP addresses into a low-level hardware address like an Ethernet address.'},
 {'question': 'What are some of the fixes and improvements made to the code according to the comments?',
  'answer': 'Some of the fixes and improvements include removing Ethernet assumptions, fixing errors in the ARP logic, adding AX25 support, adding ARP netmask code, major changes to caching and behaviour, adding Frame Relay ARP support, and converting /proc/net/arp to seq_file among others.'},
 {'question': 'What is the purpose of the \'#define pr_fmt(fmt) KBUILD_MODNAME ": " fmt\' line in the code?',
  'answer': 'This line is defining a macro for formatting print statements. It prepends the module name (KBUILD_MODNAME) and a colon to the format string for the print statement.'},
 {'question': 'What is the purpose of the arp_hash fu

In [98]:
doc_qna[3]

[{'question': 'What does the function arp_ignore do?',
  'answer': 'The arp_ignore function determines whether to ignore an ARP request based on the configuration of the incoming interface. It returns 0 to reply, 1 to ignore, or checks if the target IP is configured on the incoming interface and is in the same subnet as the source IP.'},
 {'question': 'What is the purpose of the arp_accept function?',
  'answer': 'The arp_accept function determines whether to create new entries from gratuitous ARP (GARP) based on the configuration of the incoming interface. It returns 0 to not create new entries, 1 to create new entries, or checks if the source IP is in the same subnet as an address configured on the interface that received the GARP message.'},
 {'question': 'What does the arp_filter function do?',
  'answer': 'The arp_filter function checks if the destination device of the route is the same as the incoming device. If not, it increments the LINUX_MIB_ARPFILTER statistic and sets a flag

In [37]:
qna_GT_df.to_csv("6qna_linux.csv")

In [50]:
run_evaluation(qa_rag_chain=qa_rag_chain, qna_GT_df=qna_GT_df)

/home/watson/.virtualenvs/QnAEvaluation/lib/python3.9/site-packages/langchain/evaluation/schema.py:104: UserWarning: Ignoring input in EmbeddingDistanceEvalChain, as it is not expected.
  warn(self._skip_input_warning)


,question,answer,chain_answer,labeled_criteria_grades,embedding_distance_grades,qa_llm_jugde_grades,retrieval_score
0,What is the purpose of the 'for' loop in the g...,The 'for' loop is used to calculate the factor...,The purpose of the 'for' loop in the given cod...,NaN,3.103038e-01,1,NaN
1,What does the variable 'fact' represent in the...,The variable 'fact' stores the factorial of th...,"In the given code, the variable 'fact' represe...",1.0,1.048536e-01,1,1.0
2,What is the purpose of the given code?,The purpose of the given code is to calculate ...,The purpose of the given code is to input the ...,NaN,6.754299e-01,0,1.0
3,What is the return type of the 'fact' function?,The return type of the 'fact' function is 'int'.,The return type of the 'fact' function is 'int'.,1.0,-2.220446e-16,1,1.0
4,What is the purpose of the code snippet?,To calculate and display the factorial of a nu...,The purpose of the code snippet is to prompt t...,NaN,7.435595e-01,0,NaN
5,What is the value of 'fact' after the code sni...,The factorial of the number entered by the user,The value of 'fact' cannot be determined witho...,1.0,5.335894e-01,1,1.0


In [17]:
embedding_distance_evaluator = load_evaluator(evaluator=EvaluatorType.EMBEDDING_DISTANCE,
                                              distance_metric=EmbeddingDistance.COSINE,
                                              embeddings=hf_embeddings,
                                              llm=gpt35_azure_llm)

In [21]:
embedding_distance_evaluator.evaluate_strings(
    prediction="my dog like to eat dogs food",
    reference="my dog like to eat burgers")

{'score': 0.2272324479048582}

In [12]:
-2.220446e-16

-2.220446e-16

In [9]:
from langchain.evaluation import EmbeddingDistanceEvalChain

type(embedding_distance_evaluator)
EmbeddingDistanceEvalChain

langchain.evaluation.embedding_distance.base.EmbeddingDistanceEvalChain